# Ejemplo descriptores globales para imágenes gris

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 17 de marzo de 2025

En este ejemplo se muestra algunas formas simples para obtener un descriptor de una imagen y cómo comparar descriptores.

Para este ejemplo se debe tener estas librerías instaladas:
```
pip install scipy pandas
```

Se usan estos pasos:

1. Se leen todas las imágenes jpg en la carpeta actual.

2. Se calcula un vector para cada imagen, creando una matriz de descriptores.

3. Se comparan los vectores, para esto se calcula la distancia de todos los descriptores contra todos los descriptores

4. Para cada descriptor se identifica el más cercano.

Se hacen estos pasos para distintos descriptores.

## 1. Descriptor vector de intensidades

Primero algunas funciones de ayuda:

In [ ]:
import numpy
import cv2
import os
import time

def agregar_texto(imagen, texto):
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.7
    fontThickness = 2
    position = (10, 30)
    fontColor = (50, 50, 50)
    cv2.putText(imagen, texto, position, fontFace, fontScale, fontColor, fontThickness, cv2.LINE_AA)


def mostrar_imagen(window_name, filename, imagen):
    MIN_WIDTH, MAX_WIDTH = 200, 800
    MIN_HEIGHT, MAX_HEIGHT = 200, 800
    if imagen.shape[0] > MAX_HEIGHT or imagen.shape[1] > MAX_WIDTH:
        # reducir tamaño
        fh = MAX_HEIGHT / imagen.shape[0]
        fw = MAX_WIDTH / imagen.shape[1]
        escala = min(fh, fw)
        imagen = cv2.resize(imagen, (0, 0), fx=escala, fy=escala, interpolation=cv2.INTER_CUBIC)
    elif imagen.shape[0] < MIN_HEIGHT or imagen.shape[1] < MIN_WIDTH:
        # aumentar tamaño
        fh = MIN_HEIGHT / imagen.shape[0]
        fw = MIN_WIDTH / imagen.shape[1]
        escala = max(fh, fw)
        imagen = cv2.resize(imagen, (0, 0), fx=escala, fy=escala, interpolation=cv2.INTER_NEAREST)
    # incluir el nombre sobre la imagen
    agregar_texto(imagen, filename)
    # mostrar en pantalla con el nomnbre
    cv2.imshow(window_name, imagen)


def listar_archivos_en_carpeta(imagenes_dir, extension):
    lista = []
    for archivo in os.listdir(imagenes_dir):
        # los que terminan en .jpg se agregan a la lista de nombres
        if archivo.endswith(extension):
            lista.append(archivo)
    lista.sort()
    return lista


In [ ]:
# mostrar en pantalla o no
mostrar_imagenes = True

def vector_de_intensidades(nombre_imagen, imagenes_dir):
    archivo_imagen = imagenes_dir + "/" + nombre_imagen
    imagen = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        raise Exception("no puedo abrir: " + archivo_imagen)
    # se puede cambiar a otra interpolacion, como cv2.INTER_CUBIC
    imagen_reducida = cv2.resize(imagen, (5, 5), interpolation=cv2.INTER_AREA)
    # flatten convierte una matriz de nxm en un array de largo nxm
    descriptor_imagen = imagen_reducida.flatten()
    # mostrar en pantalla, usa la variable global
    global mostrar_imagenes
    if mostrar_imagenes:
        mostrar_imagen("imagen", nombre_imagen, imagen)
        imagen_reducida_grande = cv2.resize(imagen_reducida, (imagen.shape[1],imagen.shape[0]), interpolation=cv2.INTER_NEAREST)
        mostrar_imagen("imagen_reducida_grande", nombre_imagen, imagen_reducida_grande)
        key = cv2.waitKey()
        cv2.destroyAllWindows()
        if key == ord('q') or key == 27:
            raise KeyboardInterrupt
    return descriptor_imagen


def calcular_descriptores(funcion_descriptor, nombres_imagenes, imagenes_dir):
    matriz_descriptores = None
    num_fila = 0
    t0 = time.time()
    for nombre_imagen in nombres_imagenes:
        descriptor_imagen = funcion_descriptor(nombre_imagen, imagenes_dir)
        # crear la matriz de descriptores (numero imagenes x largo_descriptor)
        if matriz_descriptores is None:
            matriz_descriptores = numpy.zeros((len(nombres_imagenes), len(descriptor_imagen)), numpy.float32)
        # copiar descriptor a una fila de la matriz de descriptores
        matriz_descriptores[num_fila] = descriptor_imagen
        num_fila += 1
    print("tiempo calcular descriptores = {:.1f} segundos".format(time.time() - t0))
    return matriz_descriptores

# cargar las imagenes de prueba
imagenes_dir = "imagenes"
nombres = listar_archivos_en_carpeta(imagenes_dir, ".jpg")
print("imágenes ({})=".format(len(nombres)))
print(nombres)
print()

descriptores = calcular_descriptores(vector_de_intensidades, nombres, imagenes_dir)

print("matriz de descriptores (filas={}, columnas={})=".format(descriptores.shape[0],
                                                               descriptores.shape[1]))

numpy.set_printoptions(linewidth=200)
print(descriptores)


### Comparar descriptores

Para comparar todos los vectores usaremos la función `cdist` de scipy. La función `cdist` recibe dos matrices y retorna la distancia entre cada una de las filas. Ambas matrices deben tener la misma cantidad de columnas (mismo largo de vectores).

Por ejemplo, si se usa `cdist(A, B)` con A una matriz de 50x100 (50 filas, 100 columnas) y B una matriz de 20x100 (20 filas, 100 columnas), retorna una matriz C de 50x20 (50 filas, 20 columnas) donde en la i-ésima fila de C está la distancia entre el i-ésimo vector de A contra las 20 filas de B.

Por defecto compara los vectores con distancia euclidiana. Se pueden cambiar la función de distancia con `metric="nombre_distancia"`. Por ejemplo, probar con `metric='cityblock'` y ver si los resultados son mejores (el más parecido de cada imagen muestra el mismo objeto).
Ver https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

In [ ]:
import scipy.spatial

matriz_distancias = scipy.spatial.distance.cdist(descriptores, descriptores, metric='euclidean')

print("matriz de distancias ({} filas, {} columnas)=".format(matriz_distancias.shape[0], matriz_distancias.shape[1]))
print()

numpy.set_printoptions(suppress=True, precision=2, linewidth=300)
print(matriz_distancias)


### Imprimir los más cercanos

Para cada imagen se muestra la imagen más cercana (que no sea sí misma).

Se usa `pandas` para mostrar los resultados como tabla.

In [ ]:
import pandas

def imprimir_cercanos(lista_nombres, matriz_distancias):
    # completar la diagonal con un valor muy grande para que el mas cercano no sea si mismo
    numpy.fill_diagonal(matriz_distancias, numpy.inf)

    # obtener la posicion del mas cercano por fila
    posiciones_minimas = numpy.argmin(matriz_distancias, axis=1)
    valores_minimos = numpy.amin(matriz_distancias, axis=1)

    tabla_resultados = []
    for i in range(len(matriz_distancias)):
        query = lista_nombres[i]
        distancia = valores_minimos[i]
        mas_cercano = lista_nombres[posiciones_minimas[i]]
        tabla_resultados.append([query, mas_cercano, distancia])

    df = pandas.DataFrame(tabla_resultados, columns=["query", "más_cercana", "distancia"])
    print(df.to_string(index=False, justify='center'))


imprimir_cercanos(nombres, matriz_distancias)


## 2. Descriptor con ecualización del histograma y  distancia euclidiana

Explicación de la ecualización del histograma: https://docs.opencv.org/4.10.0/d4/d1b/tutorial_histogram_equalization.html

In [ ]:
mostrar_imagenes = True


def vector_de_intensidades_equalizeHist(nombre_imagen, imagenes_dir):
    archivo_imagen = imagenes_dir + "/" + nombre_imagen
    imagen = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        raise Exception("no puedo abrir: " + archivo_imagen)
    # ecualizacion
    imagen = cv2.equalizeHist(imagen)
    # se puede cambiar a otra interpolacion, como cv2.INTER_CUBIC
    imagen_reducida = cv2.resize(imagen, (5, 5), interpolation=cv2.INTER_AREA)
    # flatten convierte una matriz de nxm en un array de largo nxm
    descriptor_imagen = imagen_reducida.flatten()
    # mostrar en pantalla, usa la variable global
    global mostrar_imagenes
    if mostrar_imagenes:
        mostrar_imagen("imagen", nombre_imagen, imagen)
        imagen_reducida_grande = cv2.resize(imagen_reducida, (imagen.shape[1],imagen.shape[0]), interpolation=cv2.INTER_NEAREST)
        mostrar_imagen("imagen_reducida_grande", nombre_imagen, imagen_reducida_grande)
        key = cv2.waitKey()
        cv2.destroyAllWindows()
        if key == ord('q') or key == 27:
            raise KeyboardInterrupt
    return descriptor_imagen


descriptores_equalize = calcular_descriptores(vector_de_intensidades_equalizeHist, nombres, imagenes_dir)
print(descriptores_equalize)

matriz_distancias_equalize = scipy.spatial.distance.cdist(descriptores_equalize, descriptores_equalize,
                                                          metric='euclidean')

imprimir_cercanos(nombres, matriz_distancias_equalize)

## 3. Descriptor OMD y distancia de Hamming

Reemplazar el valor gris de la zona por su posicion en un arreglo ordenado.

In [ ]:
mostrar_imagenes = True


def vector_de_intensidades_omd(nombre_imagen, imagenes_dir):
    archivo_imagen = imagenes_dir + "/" + nombre_imagen
    imagen = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        raise Exception("no puedo abrir: " + archivo_imagen)
    # se puede cambiar a otra interpolacion, como cv2.INTER_CUBIC
    imagen_reducida = cv2.resize(imagen, (5, 5), interpolation=cv2.INTER_AREA)
    # flatten convierte una matriz de nxm en un array de largo nxm
    descriptor_imagen = imagen_reducida.flatten()
    # la posicion si se ordenan
    posiciones = numpy.argsort(descriptor_imagen)
    # reemplazar el valor gris por su  posicion
    for i in range(len(posiciones)):
        descriptor_imagen[posiciones[i]] = i
    # mostrar en pantalla, usa la variable global
    global mostrar_imagenes
    if mostrar_imagenes:
        pixeles = numpy.copy(descriptor_imagen)
        for i in range(len(pixeles)):
            pixeles[i] *= 255 / (len(posiciones) - 1)
        imagen2 = pixeles.reshape((imagen_reducida.shape[1], imagen_reducida.shape[0]))
        imagen2 = cv2.resize(imagen2, (imagen.shape[1], imagen.shape[0]), interpolation=cv2.INTER_NEAREST)
        mostrar_imagen("imagen", nombre_imagen, imagen)
        mostrar_imagen("imagen_reducida_grande", nombre_imagen, imagen2)
        key = cv2.waitKey()
        cv2.destroyAllWindows()
        if key == ord('q') or key == 27:
            raise KeyboardInterrupt
    return descriptor_imagen


descriptores_omd = calcular_descriptores(vector_de_intensidades_omd, nombres, imagenes_dir)
print(descriptores_omd)

matriz_distancias_omd = scipy.spatial.distance.cdist(descriptores_omd, descriptores_omd, metric='hamming')

imprimir_cercanos(nombres, matriz_distancias_omd)


## 4. Descriptor Histogramas por zona

En este ejemplo, la imagen se está dividiendo en 4x4 zonas, y en cada zona calcula un histograma de 8 bins (en total un vector de largo 128). Probar con otras posibles divisiones.

In [ ]:
mostrar_imagenes = True


def dibujar_histograma(img, histograma, limites):
    cv2.rectangle(img, (0, 0), (img.shape[1] - 1, img.shape[0] - 1), (255, 200, 120), 1)
    pos_y_base = img.shape[0] - 6
    max_altura = img.shape[0] - 10
    nbins = len(histograma)
    for i in range(nbins):
        desde_x = int(img.shape[1] / nbins * i)
        hasta_x = int(img.shape[1] / nbins * (i + 1))
        altura = int(histograma[i] * max_altura)
        g = int((limites[i] + limites[i + 1]) / 2)
        color = (g, g, g)
        pt1 = (desde_x, pos_y_base + 5)
        pt2 = (hasta_x - 1, pos_y_base - altura)
        cv2.rectangle(img, pt1, pt2, color, -1)
    cv2.line(img, (0, pos_y_base), (img.shape[1] - 1, pos_y_base), (120, 120, 255), 1)


def histograma_por_zona(imagen, imagen_hists):
    # divisiones
    num_zonas_x = 1
    num_zonas_y = 8
    num_bins_por_zona = 16
    # procesar cada zona
    descriptor = []
    for j in range(num_zonas_y):
        desde_y = int(imagen.shape[0] / num_zonas_y * j)
        hasta_y = int(imagen.shape[0] / num_zonas_y * (j + 1))
        for i in range(num_zonas_x):
            desde_x = int(imagen.shape[1] / num_zonas_x * i)
            hasta_x = int(imagen.shape[1] / num_zonas_x * (i + 1))
            # recortar zona de la imagen
            zona = imagen[desde_y: hasta_y, desde_x: hasta_x]
            # histograma de los pixeles de la zona
            histograma, limites = numpy.histogram(zona, bins=num_bins_por_zona, range=(0, 255))
            # normalizar histograma (bins suman 1)
            histograma = histograma / numpy.sum(histograma)
            # agregar descriptor de la zona al descriptor global
            descriptor.extend(histograma)
            # dibujar histograma de la zona
            if imagen_hists is not None:
                zona_hist = imagen_hists[desde_y: hasta_y, desde_x: hasta_x]
                dibujar_histograma(zona_hist, histograma, limites)
    return descriptor


def histogramas_por_zonas(nombre_imagen, imagenes_dir):
    archivo_imagen = imagenes_dir + "/" + nombre_imagen
    imagen = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        raise Exception("no puedo abrir: " + archivo_imagen)
    # ecualizacion
    imagen = cv2.equalizeHist(imagen)
    # imagen donde se dibujan los histogramas
    global mostrar_imagenes
    imagen_hists = None
    if mostrar_imagenes:
        imagen_hists = numpy.full((imagen.shape[0], imagen.shape[1], 3), (200, 255, 200), dtype=numpy.uint8)
    descriptor = histograma_por_zona(imagen, imagen_hists)
    # mostrar imagen con histogramas
    if mostrar_imagenes:
        mostrar_imagen("imagen", nombre_imagen, imagen)
        mostrar_imagen("histograma", nombre_imagen, imagen_hists)
        key = cv2.waitKey()
        cv2.destroyAllWindows()
        if key == ord('q') or key == 27:
            raise KeyboardInterrupt
    return descriptor


descriptores_hist = calcular_descriptores(histogramas_por_zonas, nombres, imagenes_dir)

numpy.set_printoptions(suppress=True, precision=1, linewidth=100)
print(descriptores_hist)

matriz_distancias_hist = scipy.spatial.distance.cdist(descriptores_hist, descriptores_hist, metric='cityblock')

imprimir_cercanos(nombres, matriz_distancias_hist)


## 5. Descriptor HOG

Primero se aplica un filtro Gaussiano y luego filtros de Sobel para encontrar los pixeles de borde.

Sobre los pixeles de borde se calcula el ángulo del borde (entre -90 y 90 grados) y se calcula histogramas por zona de los angulos.

Está calculando la magnitud del gradiente con la fórmula:

$$magnitud = \sqrt{I_x^2 + I_y^2}$$

Se seleccionan los pixeles donde la magnitud del gradiente supere un valor de 100. Es posible probar con otros valores.

Para calcular el ángulo del gradiente está usando la función:

$$orientación = \textrm{numpy.arctan2}(I_y, I_x)$$ 

Notar que `arctan2` recibe primero el valor del eje $y$ y luego el valor del eje $x$, y que retorna un ángulo en radianes entre $[-\pi, \pi]$. Ver https://numpy.org/doc/stable/reference/generated/numpy.arctan2.html

Nunca va a suceder el caso de calcular `arctan2(0, 0)` (que es indefinido, un punto no tiene ángulo) porque el ángulo solo se calcula cuando la magnitud del gradiente supera cierto umbral.

Los histogramas se están calculando para 4x4 zonas, cada uno dividiendo el rango $[-90°, 90°]$ en 10 bins (en total 160 dimensiones). Probar con otros parámetros.

Usualmente es buena idea reducir el tamaño de la imagen o aplicar un filtro gaussiano para quitar un poco de ruido y detalles de la imagen antes de calcular los gradientes.


In [ ]:
mostrar_imagenes = True

import math

def angulos_en_zona(angulos, mascara):
    # calcular angulos de la zona
    angulos_zona = []
    for row in range(mascara.shape[0]):
        for col in range(mascara.shape[1]):
            if not mascara[row][col]:
                continue
            angulo = round(math.degrees(angulos[row][col]))
            # dejar angulos en el rango -90 a 90
            if angulo < -180 or angulo > 180:
                raise Exception("angulo invalido {}, verificar si funciona la mascara".format(angulo));
            elif angulo <= -90:
                angulo += 180
            elif angulo > 90:
                angulo -= 180
            angulos_zona.append(angulo)
    return angulos_zona


def bordes_por_zona(angulos, mascara, imagen_hists):
    # divisiones
    num_zonas_x = 4
    num_zonas_y = 4
    num_bins_por_zona = 10
    # procesar cada zona
    descriptor = []
    for j in range(num_zonas_y):
        desde_y = int(mascara.shape[0] / num_zonas_y * j)
        hasta_y = int(mascara.shape[0] / num_zonas_y * (j + 1))
        for i in range(num_zonas_x):
            desde_x = int(mascara.shape[1] / num_zonas_x * i)
            hasta_x = int(mascara.shape[1] / num_zonas_x * (i + 1))
            # calcular angulos de la zona
            angulos_zona = angulos_en_zona(angulos[desde_y: hasta_y, desde_x: hasta_x],
                                      mascara[desde_y: hasta_y, desde_x: hasta_x])
            # histograma de los angulos de la zona
            histograma, limites = numpy.histogram(angulos_zona, bins=num_bins_por_zona, range=(-90, 90))
            # normalizar histograma (bins suman 1)
            if numpy.sum(histograma) != 0:
                histograma = histograma / numpy.sum(histograma)
            # agregar descriptor de la zona al descriptor global
            descriptor.extend(histograma)
            # dibujar histograma de la zona
            if imagen_hists is not None:
                zona_hist = imagen_hists[desde_y: hasta_y, desde_x: hasta_x]
                limites = (limites + 180) / 360 * 255
                dibujar_histograma(zona_hist, histograma, limites)
    return descriptor


def histograma_bordes_por_zona(nombre_imagen, imagenes_dir):
    archivo_imagen = imagenes_dir + "/" + nombre_imagen
    imagen = cv2.imread(archivo_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        raise Exception("no puedo abrir: " + archivo_imagen)
    # calcular filtro de sobel (usa cv2.GaussianBlur para borrar ruido)
    imagen2 = cv2.GaussianBlur(imagen, (5, 5), 0, 0)
    sobelX = cv2.Sobel(imagen2, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
    sobelY = cv2.Sobel(imagen2, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
    # calcula la magnitud del gradiente en cada pixel de la imagen
    magnitud = numpy.sqrt(numpy.square(sobelX) + numpy.square(sobelY))
    # selecciona los pixeles donde la magnitud del gradiente supera un valor umbral
    threshold_magnitud_gradiente = 100
    th, imagen_bordes = cv2.threshold(magnitud, threshold_magnitud_gradiente, 255, cv2.THRESH_BINARY)
    # definir una mascara donde estan los pixeles de borde
    mascara = imagen_bordes == 255
    # para los pixeles de la mascara calcular el angulo del gradiente
    angulos = numpy.arctan2(sobelY, sobelX, where=mascara)
    # imagen donde se dibujan los histogramas
    global mostrar_imagenes
    imagen_hists = None
    if mostrar_imagenes:
        imagen_hists = numpy.full((imagen.shape[0], imagen.shape[1], 3), (200, 255, 200), dtype=numpy.uint8)
    # calcular descriptor (histograms de angulos por zonas)
    descriptor = bordes_por_zona(angulos, mascara, imagen_hists)
    # mostrar imagen con histogramas
    if mostrar_imagenes:
        mostrar_imagen("imagen", nombre_imagen, imagen)
        mostrar_imagen("bordes", "", imagen_bordes)
        mostrar_imagen("hog", "", imagen_hists)
        key = cv2.waitKey()
        cv2.destroyAllWindows()
        if key == ord('q') or key == 27:
            raise KeyboardInterrupt
    return descriptor

descriptores_bordes = calcular_descriptores(histograma_bordes_por_zona, nombres, imagenes_dir)

numpy.set_printoptions(suppress=True, precision=2, edgeitems=8, linewidth=100)
print(descriptores_bordes)

matriz_distancias_bordes = scipy.spatial.distance.cdist(descriptores_bordes, descriptores_bordes, metric='cityblock')

imprimir_cercanos(nombres, matriz_distancias_bordes)


**Pregunta:** Finalmente, ¿cuál es el mejor?

¿Cómo se puede determinar cuál es el mejor algoritmo para calcular descriptores?
